In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

import time
import math

In [2]:
df = pd.read_pickle("../data/features/analysis.pickle")

In [3]:
import re
columnNames = list(df)
regexTimes = re.compile(".*Times")
timesColumnNames = list(filter(regexTimes.match, columnNames))
for column in timesColumnNames:
    columnNames.remove(column)

In [4]:
# Remove all *Times columns
df = df[columnNames]
# Drop rows with NA
df = df.dropna()

In [5]:
minCostIndices = df[["heuristics.tabuCosts", "heuristics.simulatedAnnealingCosts", "heuristics.graspCosts", "heuristics.geneticCosts", "heuristics.antColonyCosts"]].idxmin(axis=1)

intLabels = LabelEncoder().fit_transform(minCostIndices).reshape(-1, 1)
# 5 values for 5 different heuristics
outputs = OneHotEncoder(sparse=False, n_values=5).fit_transform(intLabels)

inputs = scale(df.astype('float64'),axis=1)

inputsTrain = inputs[0:50]
outputsTrain = outputs[0:50]

inputsValid = inputs[50:]
outputsValid = outputs[50:]

In [6]:
inputsTrain.shape

(50, 43)

In [7]:
epoch_count = 0

def minibatch(batchSize, n, input_data, output_data):
    input_batches = np.empty((math.ceil(n/batchSize), batchSize) + input_data.shape[1:])
    output_batches = np.empty((math.ceil(n/batchSize), batchSize) + output_data.shape[1:])
    
    global epoch_count
    epoch_count += 1
    indexes = np.random.permutation(n)
    i = 0
    batch_i = 0
    input_array = np.zeros((batchSize,) + input_data.shape[1:])
    output_array = np.zeros((batchSize,) + output_data.shape[1:])
    for index in indexes:
        input_array[i] = input_data[index]
        output_array[i] = output_data[index]
        i += 1

        if i >= batchSize:
            input_batches[batch_i] = input_array
            output_batches[batch_i] = output_array
            i = 0
            batch_i += 1
    
    if(n % batchSize != 0):
        input_batches[batch_i] = input_array[0:i]
        output_batches[batch_i] = output_array[0:i]
    
    return (input_batches, output_batches)

In [8]:
EPOCHS = 1000

N1 = 50

NODES1 = 512
NODES2 = 256

ALPHA = 0.08

BATCH_SIZE = 50

STD = 0.1

In [11]:
# Setup Tensorflow

# Constants
x_train_full = tf.constant(inputsTrain, dtype='float32', shape=[50, 43])
y_train_full = tf.constant(outputsTrain, dtype='float32', shape=[50, 5])

x_valid_full = tf.constant(inputsValid, dtype='float32', shape=[27, 43])
y_valid_full = tf.constant(outputsValid, dtype='float32', shape=[27, 5])

x_train = tf.placeholder(tf.float32, [BATCH_SIZE, 43])
y_train = tf.placeholder(tf.float32, [BATCH_SIZE, 5])

# Variables
W_input = tf.Variable(tf.truncated_normal([43, NODES1], stddev=STD, seed = 0))
b_input = tf.Variable(tf.truncated_normal([1, NODES1], stddev=STD, seed = 0))

W_hidden = tf.Variable(tf.truncated_normal([NODES1, NODES2], stddev=STD, seed = 0))
b_hidden = tf.Variable(tf.truncated_normal([1, NODES2], stddev=STD, seed = 0))

W_hidden2 = tf.Variable(tf.truncated_normal([NODES2, 5], stddev=STD, seed = 0))
b_hidden2 = tf.Variable(tf.truncated_normal([1, 5], stddev=STD, seed = 0))

# Optimization
input_layer = tf.nn.relu(tf.matmul(x_train, W_input) + b_input)

hidden_layer = tf.nn.relu(tf.matmul(input_layer, W_hidden) + b_hidden)
hidden2_layer = tf.matmul(hidden_layer, W_hidden2) + b_hidden2

logits_train_full = tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(x_train_full, W_input) + b_input), W_hidden) + b_hidden), W_hidden2) + b_hidden2
logits_valid_full = tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(x_valid_full, W_input) + b_input), W_hidden) + b_hidden), W_hidden2) + b_hidden2

L2 = tf.reduce_mean(ALPHA * (tf.nn.l2_loss(W_input) + tf.nn.l2_loss(W_hidden) + tf.nn.l2_loss(W_hidden2)))

CE = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = hidden2_layer, labels = y_train) + ALPHA * (tf.nn.l2_loss(W_input) + tf.nn.l2_loss(W_hidden) + tf.nn.l2_loss(W_hidden2)))

CE_train_full = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits_train_full, labels = y_train_full) + ALPHA * (tf.nn.l2_loss(W_input) + tf.nn.l2_loss(W_hidden) + tf.nn.l2_loss(W_hidden2)))
CE_valid_full = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits_valid_full, labels = y_valid_full) + ALPHA * (tf.nn.l2_loss(W_input) + tf.nn.l2_loss(W_hidden) + tf.nn.l2_loss(W_hidden2)))

optimizer = tf.train.AdamOptimizer().minimize(CE)

y_pred_train = tf.nn.softmax(logits_train_full)
y_pred_valid = tf.nn.softmax(logits_valid_full)

init = tf.global_variables_initializer()

In [12]:
# Initialize TensorFlow
sess = tf.Session()
sess.run(init)

In [15]:
def printStats():
    (ce_train,ce_valid,p_train,p_valid,l2) = sess.run([CE_train_full, CE_valid_full, y_pred_train, y_pred_valid, L2])
    labels_train_pred = oneHotArray[p_train.argmax(axis=1)]
    labels_valid_pred = oneHotArray[p_valid.argmax(axis=1)]
    error_train = 0
    error_valid = 0
#     error_train = 1 - accuracy_score(CIFAR10['labels_train'], labels_train_pred)
#     error_valid = 1 - accuracy_score(CIFAR10['labels_valid'], labels_valid_pred)
    total_compute_time = (time.time() - t_start)/60
    print('%7d %7d%12.5f%12.5f%12.3f%12.3f%12f%12.1f' % (EPOCHS,epoch_count,ce_train,ce_valid,error_train,error_valid,l2,total_elapsed_time))

In [16]:
# Minimize MSE

train = True

oneHotArray = np.array([3, 5])

total_elapsed_time = 0

ce_time = 0

epoch_count = 0

print('%15s%24s%24s' % (' ','cross-entropy','error-rate'))
print('%15s%12s%12s%12s%12s%12s%12s' % ('epoch','training','validation','training','validation','L2','time (min)'))

while(train):
    batch = minibatch(BATCH_SIZE, N1, inputsTrain, outputsTrain)

    for step in range(batch[0].shape[0]):
        x_batch = batch[0][step]
        y_batch = batch[1][step]
        
        t_start = time.time()
        sess.run([optimizer], feed_dict={x_train:x_batch,y_train:y_batch})
        t_end = time.time()
        
        total_elapsed_time += (t_end - t_start)/60
        
        if t_end - ce_time > 6:
#             (ce) = sess.run(CE_train_full)
#             print("cross-entropy = %f" % (ce))
            printStats()
            
            ce_time = time.time()

            
        if epoch_count >= EPOCHS:
            train = False
            break

print("Finished")
print("Elapsed Time: %f" % (total_elapsed_time))
print("Epoch Count: %d" % (epoch_count))

printStats()

                          cross-entropy              error-rate
          epoch    training  validation    training  validation          L2  time (min)
   1000       1    47.30663    46.66875       0.000       0.000   46.377380         0.0
Finished
Elapsed Time: 0.025898
Epoch Count: 1000
   1000    1000     0.79144     0.42378       0.000       0.000    0.067083         0.0
